In [1]:
import torch

In [2]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np
import random
from scipy.special import expit

import sys
import os
import json
import re
import pickle
import time

In [4]:
from collections import Counter

def __preprocess():
    filepath = 'data/'
    # Initialize a Counter object to simplify word counting
    word_count = Counter()

    with open(filepath + 'training_label.json', 'r') as f:
        file = json.load(f)

    # Process each sentence in the data
    for d in file:
        for s in d['caption']:
            # Simplify punctuation removal and split in one step
            word_sentence = re.sub('[.!,;?]', ' ', s).lower().split()
            # Update the word counts for all words in the sentence
            word_count.update(word_sentence)

    # Filter out words with occurrences fewer than 5
    word_dict = {word: count for word, count in word_count.items() if count > 4}

    # Initial tokens for special purposes
    useful_tokens = [('<PAD>', 0), ('<SOS>', 1), ('<EOS>', 2), ('<UNK>', 3)]
    
    # Create index-to-word and word-to-index dictionaries, incorporating special tokens
    i2w = {i + len(useful_tokens): w for i, w in enumerate(word_dict)}
    w2i = {w: i + len(useful_tokens) for i, w in enumerate(word_dict)}
    for token, index in useful_tokens:
        i2w[index] = token
        w2i[token] = index

    return i2w, w2i, word_dict


In [5]:
def s_split(sentence, word_dict, w2i):
    # Normalize and split the sentence into words
    words = re.sub(r'[.!,;?]', ' ', sentence).split()

    # Convert words to their corresponding indices in w2i, defaulting to <UNK> index if not found
    indexed_sentence = [w2i.get(word, 3) for word in words]

    # Prepend <SOS> and append <EOS> tokens
    indexed_sentence = [1] + indexed_sentence + [2]

    return indexed_sentence


In [6]:
def annotate(label_file, word_dict, w2i):
    # Define the full path to the label file
    label_json_path = f'data/{label_file}'
    annotated_captions = []

    # Open and load the label file
    with open(label_json_path, 'r') as file:
        labels = json.load(file)

    # Iterate over each data entry in the loaded labels
    for data_entry in labels:
        # Process each caption using the s_split function
        for caption in data_entry['caption']:
            indexed_caption = s_split(caption, word_dict, w2i)
            annotated_captions.append((data_entry['id'], indexed_caption))

    return annotated_captions


In [7]:
def avi(files_dir):
    avi_data = {}
    training_feats = 'data/' + files_dir
    files = os.listdir(training_feats)
    i = 0
    for file in files:
        print("Loading file no:-  " + str(i))
        i+=1
        value = np.load(os.path.join(training_feats, file))
        avi_data[file.split('.npy')[0]] = value
    return avi_data

In [8]:
def minibatch(data):
    # Sort the data by the length of captions in descending order
    data.sort(key=lambda x: len(x[1]), reverse=True)
    
    # Unpack the data into separate lists of avi_data and captions
    avi_data, captions = zip(*data)
    
    # Stack the avi_data into a tensor
    avi_data = torch.stack(avi_data, 0)
    
    # Get the lengths of each caption
    lengths = [len(cap) for cap in captions]
    
    # Initialize a zero tensor for targets with dimensions [batch_size, max_caption_length]
    targets = torch.zeros(len(captions), max(lengths)).long()
    
    # Fill in the targets tensor with caption indices
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = torch.tensor(cap[:end], dtype=torch.long)
    
    # Return the avi_data, targets tensor, and lengths of each caption
    return avi_data, targets, lengths


In [9]:
class TrainingData(Dataset):
    def __init__(self, label_file, files_dir, word_dict, w2i):
        self.label_file = label_file
        self.files_dir = files_dir
        self.word_dict = word_dict
        self.avi = avi(label_file)
        self.w2i = w2i
        self.data_pair = annotate(files_dir, word_dict, w2i)
        
    def __len__(self):
        # Returns the number of items in the dataset
        return len(self.data_pair)
    
    def __getitem__(self, idx):
        #Check
        assert idx < self.__len__(), "Index out of range"
        
        avi_file_name, sentence = self.data_pair[idx]
        # Get the video data as a tensor and add some noise
        video_data = torch.Tensor(self.avi[avi_file_name])
        video_data += torch.Tensor(video_data.size()).random_(0, 2000) / 10000.
        # Convert sentence to tensor
        caption_tensor = torch.Tensor(sentence).long()
        
        return video_data, caption_tensor


In [10]:
class TestingData(Dataset):
    def __init__(self, test_data_path):
        self.avi = []
        files = os.listdir(test_data_path)
        for file in files:
            key = file.split('.npy')[0]
            value = np.load(os.path.join(test_data_path, file))
            self.avi.append([key, value])
            
    def __len__(self):
        # Returns the number of items in the dataset
        return len(self.avi)
    
    def __getitem__(self, idx):
        # Returns the idx-th item of the dataset
        assert idx < len(self), "Index out of range"
        return self.avi[idx]

In [11]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        
        self.hidden_size = hidden_size
        self.attention_layer1 = nn.Linear(2*hidden_size, hidden_size)
        self.attention_layer2 = nn.Linear(hidden_size, hidden_size)
        self.attention_layer3 = nn.Linear(hidden_size, hidden_size)
        self.attention_layer4 = nn.Linear(hidden_size, hidden_size)
        self.compute_weight = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden_state, encoder_outputs):
        batch_size, seq_len, hidden_dim = encoder_outputs.size()
        hidden_state_expanded = hidden_state.view(batch_size, 1, hidden_dim).repeat(1, seq_len, 1)
        combined_inputs = torch.cat((encoder_outputs, hidden_state_expanded), 2).view(-1, 2*self.hidden_size)

        attn_hidden = self.attention_layer1(combined_inputs)
        attn_hidden = self.attention_layer2(attn_hidden)
        attn_hidden = self.attention_layer3(attn_hidden)
        attn_hidden = self.attention_layer4(attn_hidden)
        weights = self.compute_weight(attn_hidden)
        weights = weights.view(batch_size, seq_len)
        weights_normalized = F.softmax(weights, dim=1)
        context_vector = torch.bmm(weights_normalized.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context_vector

In [12]:
class EncoderLSTM(nn.Module):
    def __init__(self):
        super(EncoderLSTM, self).__init__()
        
        self.Embedding = nn.Linear(4096, 512)
        self.dropout = nn.Dropout(0.33)
        self.lstm = nn.LSTM(512, 512, batch_first=True)

    def forward(self, input):
        batch_size, seq_len, feat_n = input.size()    
        input = input.view(-1, feat_n)
        input = self.Embedding(input)
        input = self.dropout(input)
        input = input.view(batch_size, seq_len, 512)

        output, t = self.lstm(input)
        hidden_state, context = t[0], t[1]
        return output, hidden_state

In [13]:
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size, vocab_size, word_dim, dropout_percentage=0.33):
        super(DecoderLSTM, self).__init__()

        self.hidden_size = 512
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.word_dim = word_dim

        self.embedding = nn.Embedding(output_size, 1024)
        self.dropout = nn.Dropout(0.33)
        self.lstm = nn.LSTM(hidden_size+word_dim, hidden_size, batch_first=True)
        self.attention = Attention(hidden_size)
        self.to_final_output = nn.Linear(hidden_size, output_size)


    def forward(self, encoder_last_hidden_state, encoder_output, targets=None, mode='train', tr_steps=None):
        _, batch_size, _ = encoder_last_hidden_state.size()
        
        decoder_current_hidden_state = None if encoder_last_hidden_state is None else encoder_last_hidden_state
        decoder_cxt = torch.zeros(decoder_current_hidden_state.size()).cuda()

        decoder_current_input_word = Variable(torch.ones(batch_size, 1)).long().cuda()
        seq_logProb = []
        seq_predictions = []

        targets = self.embedding(targets)
        _, seq_len, _ = targets.size()

        for i in range(seq_len-1):
            threshold = self.teacher_forcing_ratio(training_steps=tr_steps)
            if random.uniform(0.05, 0.995) > threshold: # returns a random float value between 0.05 and 0.995
                current_input_word = targets[:, i]  
            else: 
                current_input_word = self.embedding(decoder_current_input_word).squeeze(1)

            context = self.attention(decoder_current_hidden_state, encoder_output)
            lstm_input = torch.cat([current_input_word, context], dim=1).unsqueeze(1)
            lstm_output, t = self.lstm(lstm_input, (decoder_current_hidden_state,decoder_cxt))
            decoder_current_hidden_state=t[0]
            logprob = self.to_final_output(lstm_output.squeeze(1))
            seq_logProb.append(logprob.unsqueeze(1))
            decoder_current_input_word = logprob.unsqueeze(1).max(2)[1]

        seq_logProb = torch.cat(seq_logProb, dim=1)
        seq_predictions = seq_logProb.max(2)[1]
        return seq_logProb, seq_predictions
        
    def infer(self, encoder_last_hidden_state, encoder_output):
        _, batch_size, _ = encoder_last_hidden_state.size()
        decoder_current_hidden_state = None if encoder_last_hidden_state is None else encoder_last_hidden_state
        decoder_current_input_word = Variable(torch.ones(batch_size, 1)).long()
        decoder_c= torch.zeros(decoder_current_hidden_state.size())
        seq_logProb = []
        seq_predictions = []
        assumption_seq_len = 28
        
        for i in range(assumption_seq_len-1):
            current_input_word = self.embedding(decoder_current_input_word).squeeze(1)
            context = self.attention(decoder_current_hidden_state, encoder_output)
            lstm_input = torch.cat([current_input_word, context], dim=1).unsqueeze(1)
            lstm_output,  t = self.lstm(lstm_input, (decoder_current_hidden_state,decoder_c))
            decoder_current_hidden_state=t[0]
            logprob = self.to_final_output(lstm_output.squeeze(1))
            seq_logProb.append(logprob.unsqueeze(1))
            decoder_current_input_word = logprob.unsqueeze(1).max(2)[1]

        seq_logProb = torch.cat(seq_logProb, dim=1)
        seq_predictions = seq_logProb.max(2)[1]
        return seq_logProb, seq_predictions

    def teacher_forcing_ratio(self, training_steps):
        return (expit(training_steps/20 +0.85))

In [14]:
class MODELS(nn.Module):
    def __init__(self, encoder, decoder):
        super(MODELS, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, avi_feat, mode, target_sentences=None, tr_steps=None):
        encoder_outputs, encoder_last_hidden_state = self.encoder(avi_feat)
        if mode == 'train':
            seq_logProb, seq_predictions = self.decoder(encoder_last_hidden_state = encoder_last_hidden_state, encoder_output = encoder_outputs,
                targets = target_sentences, mode = mode, tr_steps=tr_steps)
        elif mode == 'inference':
            seq_logProb, seq_predictions = self.decoder.infer(encoder_last_hidden_state=encoder_last_hidden_state, encoder_output=encoder_outputs)
        return seq_logProb, seq_predictions

In [15]:
def calculate_loss(loss_fn, predictions, targets, lengths):
    batch_size = len(predictions)
    concatenated_predictions = None
    concatenated_targets = None
    is_first_batch = True 

    for i in range(batch_size):
        current_prediction = predictions[i]
        current_target = targets[i]
        current_length = lengths[i] - 1 

        # Trim the sequences based on the actual length
        trimmed_prediction = current_prediction[:current_length]
        trimmed_target = current_target[:current_length]

        # Initialize or concatenate the sequences
        if is_first_batch:
            concatenated_predictions = trimmed_prediction
            concatenated_targets = trimmed_target
            is_first_batch = False 
        else:
            concatenated_predictions = torch.cat((concatenated_predictions, trimmed_prediction), dim=0)
            concatenated_targets = torch.cat((concatenated_targets, trimmed_target), dim=0)

    # Compute the loss on the concatenated sequences
    total_loss = loss_fn(concatenated_predictions, concatenated_targets)
    avg_loss = total_loss / batch_size  #average loss

    return avg_loss


In [16]:
import torch

def minibatch(data):
    # Sort the input data by the length of the captions in descending order
    data.sort(key=lambda x: len(x[1]), reverse=True)
    video_data, captions = zip(*data)
    video_tensor = torch.stack(video_data, 0)
    
    caption_lengths = [len(cap) for cap in captions]
    
    # Initialize a zero tensor for all captions based on the longest caption
    target_tensor = torch.zeros(len(captions), max(caption_lengths)).long()
    
    # Fill in the target tensor with actual captions, padding the rest
    for index, caption in enumerate(captions):
        end = caption_lengths[index]
        target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
    
    return video_tensor, target_tensor, caption_lengths


In [17]:

def train(model, epoch, loss_fn, parameters, optimizer, train_loader):
    model.train()
    for batch_idx, batch in enumerate(train_loader):
        avi_feats, ground_truths, lengths = batch
        avi_feats, ground_truths = Variable(avi_feats).cuda(), Variable(ground_truths).cuda()
        # avi_feats, ground_truths = Variable(avi_feats), Variable(ground_truths)
        
        optimizer.zero_grad()
        seq_logProb, seq_predictions = model(avi_feats, target_sentences = ground_truths, mode = 'train', tr_steps = epoch)
        ground_truths = ground_truths[:, 1:]  
        loss = calculate_loss(loss_fn, seq_logProb, ground_truths, lengths)
        print('Batch - ', batch_idx, ' Loss - ', loss)
        loss.backward()
        optimizer.step()
        
    print("---------------------------------------------------------------------------------------------")

    loss = loss.item()
    return loss
    print(loss)

In [18]:
def test(test_loader, model, i2w):
    model.eval()
    _ss = []
    
    for batch_idx, batch in enumerate(test_loader):
     
        id, avi_feats = batch
        avi_feats = avi_feats.cuda()
        id, avi_feats = id, Variable(avi_feats).float()
        
        seq_logProb, seq_predictions = model(avi_feats, mode='inference')
        test_predictions = seq_predictions
        
        result = [[i2w[x.item()] if i2w[x.item()] != '<UNK>' else 'something' for x in s] for s in test_predictions]
        result = [' '.join(s).split('<EOS>')[0] for s in result]
        rr = zip(id, result)
        for r in rr:
            _ss.append(r)
    return _ss

In [19]:
def main():
    i2w, w2i, word_dict = __preprocess()
    with open('i2w.pickle', 'wb') as handle:
        pickle.dump(i2w, handle, protocol = pickle.HIGHEST_PROTOCOL)
    label_file = 'training_data/feat'
    files_dir = 'training_label.json'
    train_dataset = TrainingData(label_file, files_dir, word_dict, w2i)
    train_dataloader = DataLoader(dataset = train_dataset, batch_size=64, shuffle=True, num_workers=8, collate_fn=minibatch)
    
    epochs = 17
    __dropout = 0.33

    __encoder = EncoderLSTM()
    __decoder = DecoderLSTM(512, len(i2w) +4, len(i2w) +4, 1024, __dropout)
    model = MODELS(encoder=__encoder, decoder=__decoder)
    
    model = model.cuda()
    loss_function = nn.CrossEntropyLoss()
    params = model.parameters()
    optimizer = optim.Adam(params, lr=0.0001)
    loss_arr = []
    for epoch in range(epochs):
        loss = train(model, epoch+1, loss_function, params, optimizer, train_dataloader) 
        loss_arr.append(loss)

    torch.save(model, "{}/{}.h5".format('SavedModel', 'model'))
    print("Completed Training")
    
if __name__ == "__main__":
    main()

Loading file no:-  0
Loading file no:-  1
Loading file no:-  2
Loading file no:-  3
Loading file no:-  4
Loading file no:-  5
Loading file no:-  6
Loading file no:-  7
Loading file no:-  8
Loading file no:-  9
Loading file no:-  10
Loading file no:-  11
Loading file no:-  12
Loading file no:-  13
Loading file no:-  14
Loading file no:-  15
Loading file no:-  16
Loading file no:-  17
Loading file no:-  18
Loading file no:-  19
Loading file no:-  20
Loading file no:-  21
Loading file no:-  22
Loading file no:-  23
Loading file no:-  24
Loading file no:-  25
Loading file no:-  26
Loading file no:-  27
Loading file no:-  28
Loading file no:-  29
Loading file no:-  30
Loading file no:-  31
Loading file no:-  32
Loading file no:-  33
Loading file no:-  34
Loading file no:-  35
Loading file no:-  36
Loading file no:-  37
Loading file no:-  38
Loading file no:-  39
Loading file no:-  40
Loading file no:-  41
Loading file no:-  42
Loading file no:-  43
Loading file no:-  44
Loading file no:-  4

/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption

Batch -  0  Loss -  tensor(0.1202, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  1  Loss -  tensor(0.1192, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  2  Loss -  tensor(0.1187, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  3  Loss -  tensor(0.1183, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  4  Loss -  tensor(0.1175, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  5  Loss -  tensor(0.1167, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  6  Loss -  tensor(0.1158, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  7  Loss -  tensor(0.1146, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  8  Loss -  tensor(0.1138, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  9  Loss -  tensor(0.1129, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  10  Loss -  tensor(0.1122, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  11  Loss -  tensor(0.1117, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  12  Loss -  tensor(0.1098, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  

/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption

Batch -  0  Loss -  tensor(0.0595, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  1  Loss -  tensor(0.0628, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  2  Loss -  tensor(0.0582, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  3  Loss -  tensor(0.0627, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  4  Loss -  tensor(0.0618, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  5  Loss -  tensor(0.0615, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  6  Loss -  tensor(0.0591, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  7  Loss -  tensor(0.0612, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  8  Loss -  tensor(0.0595, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  9  Loss -  tensor(0.0636, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  10  Loss -  tensor(0.0666, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  11  Loss -  tensor(0.0616, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  12  Loss -  tensor(0.0609, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  

...

Batch -  0  Loss -  tensor(0.0436, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  1  Loss -  tensor(0.0421, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  2  Loss -  tensor(0.0465, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  3  Loss -  tensor(0.0417, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  4  Loss -  tensor(0.0432, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  5  Loss -  tensor(0.0414, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  6  Loss -  tensor(0.0363, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  7  Loss -  tensor(0.0452, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  8  Loss -  tensor(0.0446, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  9  Loss -  tensor(0.0467, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  10  Loss -  tensor(0.0384, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  11  Loss -  tensor(0.0439, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  12  Loss -  tensor(0.0453, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  

/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption

Batch -  0  Loss -  tensor(0.0428, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  1  Loss -  tensor(0.0424, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  2  Loss -  tensor(0.0451, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  3  Loss -  tensor(0.0435, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  4  Loss -  tensor(0.0397, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  5  Loss -  tensor(0.0440, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  6  Loss -  tensor(0.0437, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  7  Loss -  tensor(0.0403, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  8  Loss -  tensor(0.0426, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  9  Loss -  tensor(0.0408, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  10  Loss -  tensor(0.0439, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  11  Loss -  tensor(0.0390, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  12  Loss -  tensor(0.0444, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  

/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption[:end], dtype=torch.long)
/local_scratch/pbs.2062688.pbs02/ipykernel_3054952/2530721791.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor[index, :end] = torch.tensor(caption

Batch -  0  Loss -  tensor(0.0452, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  1  Loss -  tensor(0.0428, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  2  Loss -  tensor(0.0401, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  3  Loss -  tensor(0.0388, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  4  Loss -  tensor(0.0364, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  5  Loss -  tensor(0.0390, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  6  Loss -  tensor(0.0367, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  7  Loss -  tensor(0.0399, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  8  Loss -  tensor(0.0459, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  9  Loss -  tensor(0.0391, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  10  Loss -  tensor(0.0430, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  11  Loss -  tensor(0.0414, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  12  Loss -  tensor(0.0373, device='cuda:0', grad_fn=<DivBackward0>)
Batch -  